# Fatigue Analysis — Hand Calculations

Executable validation formulas for fatigue life prediction and weld fatigue.

**Reference:** `validation.md` — Fatigue, Weld-Specific

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def error_pct(ansys_val, hand_calc):
    return abs(ansys_val - hand_calc) / abs(hand_calc) * 100

def print_comparison(name, ansys_val, hand_calc, unit, threshold=5.0):
    err = error_pct(ansys_val, hand_calc)
    status = 'PASS' if err < threshold else 'FAIL'
    print(f'{name}:')
    print(f'  Hand calc:  {hand_calc:.4f} {unit}')
    print(f'  ANSYS:      {ansys_val:.4f} {unit}')
    print(f'  Error:      {err:.2f}% [{status} — threshold {threshold}%]')
    print()

---
## 1. Modified Goodman Diagram

In [ ]:
# --- INPUT ---
S_ut = 600e6    # Ultimate tensile strength [Pa]
S_e = 250e6     # Endurance limit (corrected) [Pa]

# Applied loading
sigma_max = 300e6   # Max stress in cycle [Pa]
sigma_min = 50e6    # Min stress in cycle [Pa]

# --- HAND CALC ---
sigma_a = (sigma_max - sigma_min) / 2    # Alternating stress
sigma_m = (sigma_max + sigma_min) / 2    # Mean stress

# Goodman: σ_a/S_e + σ_m/S_ut = 1/n
n_goodman = 1 / (sigma_a / S_e + sigma_m / S_ut)

print(f'σ_max = {sigma_max/1e6:.0f} MPa')
print(f'σ_min = {sigma_min/1e6:.0f} MPa')
print(f'σ_a = (σ_max - σ_min)/2 = {sigma_a/1e6:.0f} MPa')
print(f'σ_m = (σ_max + σ_min)/2 = {sigma_m/1e6:.0f} MPa')
print(f'\nGoodman safety factor: n = {n_goodman:.3f}')
print(f'→ {"SAFE (n > 1)" if n_goodman > 1 else "FAILURE PREDICTED (n < 1)"}')

# --- PLOT ---
fig, ax = plt.subplots(figsize=(8, 6))

# Goodman line
ax.plot([0, S_ut/1e6], [S_e/1e6, 0], 'b-', linewidth=2, label='Goodman line')
# Yield line
S_y = 0.85 * S_ut  # Approximate
ax.plot([0, S_y/1e6], [S_y/1e6, 0], 'r--', linewidth=1, label='Yield line')

# Operating point
ax.plot(sigma_m/1e6, sigma_a/1e6, 'ro', markersize=10, label=f'Operating point (n={n_goodman:.2f})')

# Load line from origin through operating point
slope = sigma_a / sigma_m if sigma_m > 0 else np.inf
if sigma_m > 0:
    m_range = np.linspace(0, S_ut/1e6, 100)
    ax.plot(m_range, slope * m_range, 'g--', alpha=0.5, label='Load line')

ax.set_xlabel('Mean Stress σ_m (MPa)')
ax.set_ylabel('Alternating Stress σ_a (MPa)')
ax.set_title('Modified Goodman Diagram')
ax.set_xlim(0, S_ut/1e6 * 1.1)
ax.set_ylim(0, S_e/1e6 * 1.5)
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.savefig('../results/goodman_diagram.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 2. Endurance Limit with Marin Factors

In [ ]:
# --- INPUT ---
S_ut = 600e6    # Ultimate tensile strength [Pa]

# Marin factors (Shigley Ch. 6)
k_a = 0.0       # Surface finish (set to 0 to auto-compute)
k_b = 0.0       # Size factor (set to 0 to auto-compute)
k_c = 1.0       # Loading: 1.0 (bending), 0.85 (axial), 0.59 (torsion)
k_d = 1.0       # Temperature factor
k_e = 0.897     # Reliability: 0.897 (90%), 0.814 (99%), 0.753 (99.9%)

# For auto-compute
surface = 'machined'   # 'ground', 'machined', 'hot_rolled', 'forged'
d = 0.04               # Diameter [m] (for size factor)

# --- HAND CALC ---
# Uncorrected endurance limit
S_e_prime = 0.5 * S_ut if S_ut <= 1400e6 else 700e6

# Surface factor k_a = a * S_ut^b (Shigley Table 6-2)
surface_params = {
    'ground':      (1.58,  -0.085),
    'machined':    (4.51,  -0.265),
    'hot_rolled':  (57.7,  -0.718),
    'forged':      (272,   -0.995),
}
if k_a == 0:
    a, b = surface_params[surface]
    k_a = a * (S_ut / 1e6)**b
    k_a = min(k_a, 1.0)

# Size factor k_b (Shigley Eq. 6-19, 6-20)
if k_b == 0:
    d_mm = d * 1000
    if d_mm <= 2.79:
        k_b = 1.0
    elif d_mm <= 51:
        k_b = 1.24 * d_mm**(-0.107)
    elif d_mm <= 254:
        k_b = 1.51 * d_mm**(-0.157)
    else:
        k_b = 0.6

S_e = k_a * k_b * k_c * k_d * k_e * S_e_prime

print(f"S'_e = 0.5·S_ut = {S_e_prime/1e6:.0f} MPa")
print(f'\nMarin factors:')
print(f'  k_a (surface, {surface}) = {k_a:.3f}')
print(f'  k_b (size, d={d*1000:.0f}mm) = {k_b:.3f}')
print(f'  k_c (loading) = {k_c:.3f}')
print(f'  k_d (temperature) = {k_d:.3f}')
print(f'  k_e (reliability) = {k_e:.3f}')
print(f'\nS_e = k_a·k_b·k_c·k_d·k_e·S\'_e = {S_e/1e6:.1f} MPa')

---
## 3. S-N Curve Life Prediction

In [ ]:
# --- INPUT ---
S_ut = 600e6    # Ultimate tensile strength [Pa]
S_e = 200e6     # Corrected endurance limit [Pa] (from Section 2)
sigma_rev = 350e6   # Fully reversed stress amplitude [Pa]

ansys_life = 0      # <-- Enter ANSYS predicted life [cycles]

# --- HAND CALC ---
# S-N curve: S = a * N^b (Shigley Ch. 6)
# At N=1000: S_1000 = f * S_ut (f ≈ 0.9 for bending)
f = 0.9
S_1000 = f * S_ut

# Solve for a, b in S = a * N^b
# At N=1000: S_1000 = a * 1000^b
# At N=1e6: S_e = a * 1e6^b
b_sn = np.log10(S_1000 / S_e) / (np.log10(1000) - np.log10(1e6))
a_sn = S_1000 / 1000**b_sn

# Life prediction
if sigma_rev > S_e:
    N = (sigma_rev / a_sn)**(1 / b_sn)
    print(f'S-N curve: S = {a_sn/1e6:.1f} × N^({b_sn:.4f})')
    print(f'\nAt σ_rev = {sigma_rev/1e6:.0f} MPa:')
    print(f'  Predicted life: N = {N:.0f} cycles')
    print(f'  = {N:.2e} cycles')
else:
    print(f'σ_rev = {sigma_rev/1e6:.0f} MPa < S_e = {S_e/1e6:.0f} MPa')
    print('→ Infinite life (below endurance limit)')
    N = np.inf

print()
if ansys_life > 0 and np.isfinite(N):
    # For fatigue, within factor of 2 is acceptable
    ratio = ansys_life / N
    print(f'ANSYS life: {ansys_life:.0f} cycles')
    print(f'Hand calc: {N:.0f} cycles')
    print(f'Ratio: {ratio:.2f}')
    print(f'→ {"PASS (within factor of 2)" if 0.5 < ratio < 2.0 else "FAIL (outside factor of 2)"}')

# --- PLOT S-N CURVE ---
N_range = np.logspace(2, 7, 200)
S_range = a_sn * N_range**b_sn
S_range = np.clip(S_range, 0, S_ut)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(N_range, S_range/1e6, 'b-', linewidth=2)
ax.axhline(y=S_e/1e6, color='g', linestyle='--', label=f'S_e = {S_e/1e6:.0f} MPa')
if np.isfinite(N):
    ax.plot(N, sigma_rev/1e6, 'ro', markersize=10, label=f'σ = {sigma_rev/1e6:.0f} MPa → N = {N:.0f}')
ax.set_xlabel('Cycles to Failure N')
ax.set_ylabel('Stress Amplitude S (MPa)')
ax.set_title('S-N Curve')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.savefig('../results/sn_curve.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Weld Fatigue — IIW FAT Classification

In [ ]:
# --- INPUT ---
FAT = 71        # IIW FAT class [MPa] (e.g., 71 for transverse butt weld)
                # Common FAT classes (IIW):
                #   160: rolled/extruded base material (sharp edges removed)
                #   80:  transverse butt weld, ground flush
                #   71:  transverse butt weld, as-welded
                #   63:  transverse fillet weld (load-carrying)
                #   50:  cruciform joint with fillet welds
                #   36:  root crack in fillet-welded T/cruciform joint

sigma_range = 120   # Applied stress range Δσ [MPa]
m = 3               # S-N slope (IIW default = 3 for normal stress)

ansys_weld_life = 0     # <-- Enter ANSYS fatigue life at weld [cycles]

# --- HAND CALC ---
# IIW S-N: N = (FAT / Δσ)^m × 2e6
# At N = 2e6 cycles, allowable stress range = FAT
N_ref = 2e6
N = (FAT / sigma_range)**m * N_ref

# Constant amplitude fatigue limit (CAFL) — knee point at 1e7 cycles (IIW)
CAFL = FAT * (N_ref / 1e7)**(1/m)

print(f'FAT class: {FAT} MPa')
print(f'Applied stress range: Δσ = {sigma_range} MPa')
print(f'S-N slope: m = {m}')
print(f'\nPredicted life: N = (FAT/Δσ)^m × 2×10⁶')
print(f'  N = ({FAT}/{sigma_range})^{m} × 2×10⁶ = {N:.0f} cycles')
print(f'  = {N:.2e} cycles')
print(f'\nCAFL (at 10⁷): {CAFL:.1f} MPa')

if sigma_range < CAFL:
    print('→ Below CAFL — infinite life (constant amplitude loading)')
print()

if ansys_weld_life > 0:
    ratio = ansys_weld_life / N
    print(f'ANSYS life: {ansys_weld_life:.0f} cycles')
    print(f'IIW prediction: {N:.0f} cycles')
    print(f'Ratio: {ratio:.2f}')
    print(f'→ {"PASS" if 0.5 < ratio < 2.0 else "FAIL"} (factor of 2 criterion)')

---
## 5. Weld Stress — Fillet Weld Under Combined Loading

In [ ]:
# --- INPUT ---
F = 20000       # Direct shear force [N]
M = 500         # Bending moment at weld group [N·m]
h = 0.006       # Weld leg size [m]
l = 0.1         # Weld length (each side) [m]
d = 0.08        # Distance between welds (centroid separation) [m]
n_welds = 2     # Number of parallel welds

# --- HAND CALC ---
throat = 0.707 * h
A_total = n_welds * throat * l                      # Total throat area
I_weld = n_welds * throat * l * (d/2)**2            # Moment of inertia of weld group

# Primary shear (direct)
tau_prime = F / A_total

# Secondary shear (from moment)
r = d / 2                                           # Max distance from centroid
tau_double_prime = M * r / I_weld

# Combined (vector sum — worst case is when they add)
tau_max = tau_prime + tau_double_prime               # Conservative: direct addition
tau_resultant = np.sqrt(tau_prime**2 + tau_double_prime**2)  # If perpendicular

print(f'Throat = 0.707 × {h*1000:.0f} mm = {throat*1000:.2f} mm')
print(f'A_total = {A_total*1e6:.1f} mm²')
print(f'I_weld = {I_weld:.6e} m⁴')
print(f"\nτ' (direct shear) = F/A = {tau_prime/1e6:.1f} MPa")
print(f"τ'' (moment shear) = Mr/I = {tau_double_prime/1e6:.1f} MPa")
print(f'\nτ_max (conservative add) = {tau_max/1e6:.1f} MPa')
print(f'τ_resultant (vector sum) = {tau_resultant/1e6:.1f} MPa')

# Allowable
S_ut_electrode = 482e6     # E70xx
tau_allow = 0.30 * S_ut_electrode
SF = tau_allow / tau_max
print(f'\nAllowable τ (E70) = {tau_allow/1e6:.1f} MPa')
print(f'Safety factor = {SF:.2f}')
print(f'→ {"SAFE" if SF > 1 else "UNSAFE"}')